# Relatório - Desafio Casa dos Ventos - GED-20

### Etapas

__1. Sample__: importação e geração das bases dos dados que serão utilizadas;

__2. Explore__: análise exploratória dos dados;

__3. Modify__: transformação dos dados (geração de novas features) e divisão dos dados em conjuntos de treino e teste dos modelos.

### Sample

#### Importando as bibliotecas

In [ ]:
import pandas as pd
from data_import import *

__data_import.py__

```python
import pandas as pd

def separate_plants_ts():
    merge = pd.read_csv("../data/merge.csv")
    for code in merge["ana_code"].unique():
        path = f"../data/ts-{code}.csv"
        ts = merge[merge["ana_code"] == code].sort_values(by="date_ref").reset_index(drop=True)
        ts.index = pd.to_datetime(ts["date_ref"])
        ts = ts.drop(["date_ref", "ana_code"], axis=1)
        ts.to_csv(path)

def get_plants():
    df = pd.read_csv("../data/merge.csv")
    return  df["ana_code"].unique()

def import_precipitation_ts():
    df_dict = {}
    for plant in get_plants():
        df_dict[plant] = pd.read_csv(f"../data/ts-{plant}.csv", index_col=0)
    return df_dict
```

#### Gerando os dados a partir de merge.csv

In [ ]:
separate_plants_ts()

#### Importando os dados

In [ ]:
ts_dict = import_precipitation_ts()
plants = get_plants()

In [ ]:
plants

### Explore

Usinas selecionadas:

- Grande: Furnas- PSATFUR

- Paranaíba: Emborcação- PSATEMB ou Itumbiara- PSATIMBR

- Paraná: Itaipu- PSATITP

- Iguaçu: Foz do Areia (ou G. B. Munhoz)- PSATFZA

- Uruguai: Campos Novos- PSATCNV

- Tocantins: Lajeado- PSATLAJ

- Madeira: Jirau- PSATJIRA

- Xingu: Pimental- PSATPIME

#### Importando bibliotecas

In [ ]:
from data_analysis import *

__data_analysis.py__

```python
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image

def plot_plant_ts_daily(ts_dict, plant="PSATCNV", close=True):
    plt.figure(figsize=(8,6))
    ts_dict[plant].plot()
    plt.title(f"Precipitação média diária para {plant}")
    folder_path = f"figs-{plant}/"
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
    save_path = f"{folder_path}daily_mean.png"
    plt.savefig(save_path)
    if close:
        plt.close()
    return save_path
       
def plot_plant_ts_accumulated_by_month(ts_dict, plant="PSATCNV", close=True):
    plt.figure(figsize=(8, 6))
    ts = ts_dict[plant]
    df = ts.copy()
    df.index = pd.to_datetime(df.index)
    df = df.resample('M').sum()
    df.plot(color="darkblue", legend=False)
    plt.title(f"Precipitação média acumulada por mês para {plant}")
    plt.xlabel("Data de referência")
    folder_path = f"figs-{plant}/"
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
    save_path = f"{folder_path}accumulated_by_month.png"
    plt.savefig(save_path)
    if close:
        plt.close()
    return save_path
    
def plot_plant_ts_accumulated_by_year(ts_dict, plant="PSATCNV", close=True):
    plt.figure(figsize=(8,6))
    ts = ts_dict[plant]
    df = ts.copy()
    df.index = pd.to_datetime(df.index)
    df = df[df.index < "2024-01-01"]
    df = df.resample('Y').sum()
    df.plot(color="darkblue", legend=False)
    plt.title(f"Precipitação média acumulada por ano para {plant}")
    plt.xlabel("Data de referência")
    plt.grid()
    folder_path = f"figs-{plant}/"
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
    save_path = f"{folder_path}accumulated_by_year.png"
    plt.savefig(save_path)
    if close:
        plt.close()
    return save_path
    
def plot_plant_ts_mean_by_month(ts_dict, plant="PSATCNV", close=True):
    plt.figure(figsize=(8,6))
    ts = ts_dict[plant]
    df = ts.copy()
    df.index = pd.to_datetime(df.index)
    df = df.resample('M').sum()
    df = df.groupby(df.index.month).mean()
    plt.bar(df.index, df["mean_precipitation"], color="darkblue")
    plt.xticks([i+1 for i in range(len(df.index))], list(df.index))
    plt.title(f"Média das precipitações acumuladas por mês para {plant} ao longo dos anos")
    plt.xlabel("Data de referência", fontsize=12)
    folder_path = f"figs-{plant}/"
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
    save_path = f"{folder_path}mean-by-month.png"
    plt.savefig(save_path)
    if close:
        plt.close()
    return save_path

def precipitation_plots(ts_dict, plant="PSATCNV"):
    save_path0 = plot_plant_ts_daily(ts_dict, plant)
    save_path1 = plot_plant_ts_accumulated_by_month(ts_dict, plant)
    save_path2 = plot_plant_ts_accumulated_by_year(ts_dict, plant)
    save_path3 = plot_plant_ts_mean_by_month(ts_dict, plant)
    fig, axs= plt.subplots(2, 2, figsize=(20, 18))
    img0 = Image.open(save_path0) 
    img1 = Image.open(save_path1)
    img2 = Image.open(save_path2)
    img3 = Image.open(save_path3)
    axs[0, 0].imshow(img0)
    axs[0, 1].imshow(img1)
    axs[1, 0].imshow(img2)
    axs[1, 1].imshow(img3)
    for ax in axs.flatten():
        ax.axis("off")
    folder_path = f"figs-{plant}/"
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
    save_path = f"{folder_path}all.png"
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    plt.show()
```

#### Análise das séries temporais a partir dos gráficos

__PSATFUR - Furnas__

In [ ]:
df = ts_dict["PSATFUR"]

In [ ]:
df.shape

In [ ]:
begin_date = df.head(1).index[0]
end_date = df.tail(1).index[0]

print(f"Data de início das medições: {begin_date}")
print(f"Data de fim das medições: {end_date}")

In [ ]:
df.describe()

Por meio do gráfico de precipitação média acumulada por ano, verifica-se tendência decrescente de chuvas entre 2009 e 2014, bem como tendência de crescimento entre 2015 e 2023. 

Além disso, por meio do gráfico de barras dos meses, nota-se que a maior quantidade de chuvas ocorre no começo e no fim do ano, de outubro a dezembro e de janeiro a março.

In [ ]:
precipitation_plots(ts_dict, "PSATFUR")

__PSATIMBR - Itumbiara__

In [ ]:
df = ts_dict["PSATIMBR"]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
begin_date = df.head(1).index[0]
end_date = df.tail(1).index[0]

print(f"Data de início das medições: {begin_date}")
print(f"Data de fim das medições: {end_date}")

In [ ]:
df.describe()

Por meio do gráfico de precipitação média acumulada por ano, verifica-se tendência de manutenção da quantidade de chuvas ao longo dos anos, com poucas variações médias entre eles.

Além disso, por meio do gráfico de barras dos meses, nota-se que a maior quantidade de chuvas ocorre no começo e no fim do ano, de outubro a dezembro e de janeiro a março.

In [ ]:
precipitation_plots(ts_dict, "PSATIMBR")

__PSATITP - Itaipu__

In [ ]:
df = ts_dict["PSATITP"]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
begin_date = df.head(1).index[0]
end_date = df.tail(1).index[0]

print(f"Data de início das medições: {begin_date}")
print(f"Data de fim das medições: {end_date}")

In [ ]:
df.describe()

Por meio do gráfico de precipitação média acumulada por ano, verifica-se tendência de manutenção da quantidade de chuvas, no geral. No entanto, nota-se que em 2008 e em 2020, a quantidade de chuvas foi abaixo do normal.

Além disso, por meio do gráfico de barras dos meses, nota-se que a maior quantidade de chuvas ocorre nos meses de outubro a fevereiro, mas principalmente no mês de outubro.

In [ ]:
precipitation_plots(ts_dict, "PSATITP")

__PSATFZA - Foz do Areia__

In [ ]:
df = ts_dict["PSATFZA"]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
begin_date = df.head(1).index[0]
end_date = df.tail(1).index[0]

print(f"Data de início das medições: {begin_date}")
print(f"Data de fim das medições: {end_date}")

In [ ]:
df.describe()

Por meio do gráfico de precipitação média acumulada por ano, verifica-se tendência de manutenção da quantidade de chuvas, no geral. No entanto, nota-se que em 2012 e em 2021, a quantidade de chuvas foi abaixo do normal, e que em 2015 foi bem acima do normal.

Além disso, por meio do gráfico de barras dos meses, nota-se que a maior quantidade de chuvas ocorre no mês de outubro, com boa constância na quantidade de chuvas ao longo de todos os meses do ano.

In [ ]:
precipitation_plots(ts_dict, "PSATFZA")

__PSATCNV - Campos Novos__

In [ ]:
df = ts_dict["PSATCNV"]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
begin_date = df.head(1).index[0]
end_date = df.tail(1).index[0]

print(f"Data de início das medições: {begin_date}")
print(f"Data de fim das medições: {end_date}")

In [ ]:
df.describe()

Por meio do gráfico de precipitação média acumulada por ano, verifica-se tendência de manutenção da quantidade de chuvas, no geral. No entanto, nota-se que em 2003, 2004, 2012 e 2020, a quantidade de chuvas foi bem abaixo do normal, e que em 2011, 2014 e 2015 foi bem acima do normal.

Além disso, por meio do gráfico de barras dos meses, nota-se que a maior quantidade de chuvas ocorre no mês de outubro, com boa constância na quantidade de chuvas ao longo de todos os meses do ano.

In [ ]:
precipitation_plots(ts_dict, "PSATCNV")

__PSATLAJ - Lajeado__

In [ ]:
df = ts_dict["PSATLAJ"]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
begin_date = df.head(1).index[0]
end_date = df.tail(1).index[0]

print(f"Data de início das medições: {begin_date}")
print(f"Data de fim das medições: {end_date}")

In [ ]:
df.describe()

Por meio do gráfico de precipitação média acumulada por ano, verifica-se tendência de manutenção da quantidade de chuvas ao longo dos anos, com poucas variações médias entre eles.

Além disso, por meio do gráfico de barras dos meses, nota-se que a maior quantidade de chuvas ocorre no começo e no fim do ano, de outubro a dezembro e de janeiro a março. Ademais, nota-se grande diminuição na quantidade de chuvas no meio do ano, nos meses 6, 7 e 8, principalmente.

In [ ]:
precipitation_plots(ts_dict, "PSATLAJ")

__PSATJIRA - Jirau__

In [ ]:
df = ts_dict["PSATJIRA"]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
begin_date = df.head(1).index[0]
end_date = df.tail(1).index[0]

print(f"Data de início das medições: {begin_date}")
print(f"Data de fim das medições: {end_date}")

In [ ]:
df.describe()

Por meio do gráfico de precipitação média acumulada por ano, verifica-se tendência de manutenção da quantidade de chuvas ao longo dos anos, com poucas variações médias entre eles.

Além disso, por meio do gráfico de barras dos meses, nota-se que a maior quantidade de chuvas ocorre no começo e no fim do ano, de outubro a dezembro e de janeiro a março. Ademais, nota-se grande diminuição na quantidade de chuvas no meio do ano, nos meses 6, 7 e 8, principalmente.

In [ ]:
precipitation_plots(ts_dict, "PSATJIRA")

__PSATPIME - Pimental__

In [ ]:
df = ts_dict["PSATPIME"]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
begin_date = df.head(1).index[0]
end_date = df.tail(1).index[0]

print(f"Data de início das medições: {begin_date}")
print(f"Data de fim das medições: {end_date}")

In [ ]:
df.describe()

Por meio do gráfico de precipitação média acumulada por ano, verifica-se tendência de manutenção da quantidade de chuvas ao longo dos anos, com poucas variações médias entre eles.

Além disso, por meio do gráfico de barras dos meses, nota-se que a maior quantidade de chuvas ocorre principalmente no começo ano, de janeiro a março. Ademais, nota-se grande diminuição na quantidade de chuvas no meio do ano, nos meses 6, 7 e 8, principalmente, e tendência de aumento gradativo a partir de setembro.

In [ ]:
precipitation_plots(ts_dict, "PSATPIME")

### Modify

```python
def separate_predictions_plants_ts(name: str):
    df = pd.read_csv(f"../data/{name}.csv")
    df['date_ref'] = pd.to_datetime(df['date_ref'])
    df['date_forecast'] = pd.to_datetime(df['date_forecast'])
    df['date_diff'] = (df['date_forecast'] - df['date_ref']).dt.days

    pivot_df = df.pivot_table(index=['ana_code', 'date_ref'],
                              columns='date_diff',
                              values='mean_precipitation',
                              fill_value=0)
    transformed_df = pivot_df.reset_index()
    transformed_df.columns.name = None  # Remove the column name (date_diff)
    days_columns = [f'{name}_d+{col}' for col in transformed_df.columns[2:]]
    transformed_df.columns = ['ana_code', 'date_ref'] + days_columns

    for code in transformed_df["ana_code"].unique():
        path = f"../data/ts-{code}.csv"
        ts = pd.read_csv(path)
        ts['date_ref'] = pd.to_datetime(ts['date_ref'])
        code_df = transformed_df[transformed_df["ana_code"] == code].drop(columns='ana_code')
        merged = pd.merge(code_df, ts, on='date_ref', how='inner')
        merged.to_csv(path, index=False)
```

In [ ]:
import pandas as pd
from data_import import *

In [ ]:
separate_predictions_plants_ts('cfs')
separate_predictions_plants_ts('gefs')

#### Geração de Features a partir dos dados de precipitação

Nesta parte, criaremos novas features: a precipitação média em cada um dos últimos 15 dias, a precipitação total dos últimos 15 e dos últimos 30 dias
```python
def create_new_features(ts_dict: dict):
    for key, df in ts_dict.items():

        # Calculando as datas anteriores e os valores correspondentes
        for i in range(1, 16):
            df[f'-{i}d'] = df['mean_precipitation'].shift(i)

        ts_dict[key] = df
        df['mean_last_15d'] = df['mean_precipitation'].rolling(window=15).sum()
        df['mean_last_30d'] = df['mean_precipitation'].rolling(window=30).sum()


```

In [ ]:
create_new_features(ts_dict)

#### Separação dos dados de treino e teste

```python
def split_train_test(ts_dict: dict):
    X_train_dict = {}
    X_test_dict = {}
    y_train_dict = {}
    y_test_dict = {}

    for key, df in ts_dict.items():
        X = df.drop(columns=['mean_precipitation'])
        y = df['mean_precipitation']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

        X_train_dict[key] = X_train
        X_test_dict[key] = X_test
        y_train_dict[key] = y_train
        y_test_dict[key] = y_test

    return X_train_dict, X_test_dict, y_train_dict, y_test_dict
```

In [ ]:
X_train_dict, X_test_dict, y_train_dict, y_test_dict = split_train_test(ts_dict)